In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Anomaly detection models
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import LogisticRegression

# Preprocessing and evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

# For handling extreme imbalance
# Estimates class weight for the minority class, so that the model can learn well from minority class.
from sklearn.utils.class_weight import compute_class_weight


In [17]:
# Load data and explore the class imbalance

# Loading fire and weather data from parquet files.
fire_data = pd.read_parquet('final_fire_df.parquet')
weather_data = pd.read_parquet('daily_weather.parquet')

print(f"Fire data: {fire_data.shape[0]:,} records")
print(f"Weather data: {weather_data.shape[0]:,} records")

# Exploring the extreme imbalance issue
# Checks the current seasonal distribution of fires.
seasonal_dist = fire_data['season'].value_counts()
print(f"\nSeasonal Distribution:")
for season, count in seasonal_dist.items():
    pct = count/len(fire_data)*100
    print(f"  {season}: {count:,} ({pct:.1f}%)")

# Check monthly distribution of fires for winter months
winter_fires = fire_data[fire_data['acq_date'].dt.month.isin([6, 7, 8])]
print(f"\nWinter fires (Jun-Aug): {len(winter_fires):,} total")

# Check FRP distribution
print(f"\nFRP Statistics:")
print(f"  Mean FRP: {fire_data['frp'].mean():.2f}")
print(f"  75th percentile: {fire_data['frp'].quantile(0.75):.2f}")
print(f"  90th percentile: {fire_data['frp'].quantile(0.90):.2f}")
print(f"  95th percentile: {fire_data['frp'].quantile(0.95):.2f}")

# Our Current definition of unusual fire is 'Top 25% FRP in winter'
current_threshold = fire_data['frp'].quantile(0.75)
current_outliers = fire_data[
    (fire_data['frp'] > current_threshold) & 
    (fire_data['acq_date'].dt.month.isin([6, 7, 8]))
]

print(f"Current Anomaly Definition Results:")
print(f"Threshold: FRP > {current_threshold:.2f} in winter months")
print(f"Anomalous fires: {len(current_outliers)}")
print(f"Percentage: {len(current_outliers)/len(fire_data)*100:.3f}%")

if len(current_outliers) <= 15:
    print(f"EXTREMELY LOW OUTLIERS - Need better definition!")
    
    # Trying alternative definitions
    print(f"Testing Alternative Definitions:")
    
    # Alternative 1: Top 10% FRP in winter
    alt1_threshold = fire_data['frp'].quantile(0.90)
    alt1_outliers = fire_data[
        (fire_data['frp'] > alt1_threshold) & 
        (fire_data['acq_date'].dt.month.isin([6, 7, 8]))
    ]
    print(f"Top 10% FRP in winter: {len(alt1_outliers)} samples")
    
    # Alternative 2: High FRP in low fire season (May-Aug)
    alt2_threshold = fire_data['frp'].quantile(0.80)
    alt2_outliers = fire_data[
        (fire_data['frp'] > alt2_threshold) & 
        (fire_data['acq_date'].dt.month.isin([5, 6, 7, 8]))
    ]
    print(f"Top 20% FRP in low season: {len(alt2_outliers)} samples")
    
    # Alternative 3: Very high FRP anywhere (top 5%)
    alt3_threshold = fire_data['frp'].quantile(0.95)
    alt3_outliers = fire_data[fire_data['frp'] > alt3_threshold]
    print(f"Top 5% FRP anywhere: {len(alt3_outliers)} samples")


Fire data: 8,140 records
Weather data: 7,665 records

Seasonal Distribution:
  Autumn: 7,312 (89.8%)
  Summer: 437 (5.4%)
  Spring: 344 (4.2%)
  Winter: 47 (0.6%)

Winter fires (Jun-Aug): 47 total

FRP Statistics:
  Mean FRP: 33.69
  75th percentile: 32.80
  90th percentile: 79.54
  95th percentile: 122.10
Current Anomaly Definition Results:
Threshold: FRP > 32.80 in winter months
Anomalous fires: 10
Percentage: 0.123%
EXTREMELY LOW OUTLIERS - Need better definition!
Testing Alternative Definitions:
Top 10% FRP in winter: 5 samples
Top 20% FRP in low season: 80 samples
Top 5% FRP anywhere: 407 samples


In [21]:
def create_anomaly_dataset(anomaly_definition='flexible'):
    """
    Creates a dataset for anomaly detection with our flexible anomaly definition
    """
    
    # Taking a manageable sample for faster training time.
    # I initially planned to take a sample of 3000, but decided to use all the data
    # since I found lesser anomalies.
    fire_sample = fire_data.sample(n=len(fire_data), random_state=42)
    print(f"Working with {len(fire_sample):,} fire samples")
    
    # Creating the combined dataset
    combined_data = []
    
    for idx, fire in fire_sample.iterrows():
        fire_date = fire['acq_date'].date()
        fire_lat, fire_lon = fire['latitude'], fire['longitude']
        
        # Finding weather data for same date
        weather_day = weather_data[weather_data['date'].dt.date == fire_date]
        
        if not weather_day.empty:
            # Getting the nearest weather station using euclidean distance.
            weather_day = weather_day.copy()
            weather_day['distance'] = ((weather_day['latitude'] - fire_lat)**2 + 
                                     (weather_day['longitude'] - fire_lon)**2)**0.5
            closest_weather = weather_day.loc[weather_day['distance'].idxmin()]
            
            # Creates feature record
            record = {
                # Fire characteristics
                'frp': fire['frp'],
                'brightness': fire['brightness'],
                'confidence': 1 if fire['confidence'] == 'h' else 0,
                'month': fire['acq_date'].month,
                
                # Weather features
                'temp_max': closest_weather['temperature_2m_max'],
                'temp_min': closest_weather['temperature_2m_min'],
                'humidity_min': closest_weather['relative_humidity_2m_min'],
                'humidity_max': closest_weather['relative_humidity_2m_max'],
                'wind_speed': closest_weather['wind_speed_10m_max'],
                'precipitation': closest_weather['precipitation_sum'],
                'solar_radiation': closest_weather['shortwave_radiation_sum'],
                
                # Feature Engineering
                'temp_range': closest_weather['temperature_2m_max'] - closest_weather['temperature_2m_min'],
                'humidity_range': closest_weather['relative_humidity_2m_max'] - closest_weather['relative_humidity_2m_min'],
                
                # Date of the fire
                'date': fire_date
            }
            combined_data.append(record)
    
    dataset = pd.DataFrame(combined_data)
    
    # Defining anomalies based on the chosen definition
    if anomaly_definition == 'flexible':
        # Choose definition that gives us 30-100 anomalies for better modeling
        
        # Test different thresholds
        frp_95 = dataset['frp'].quantile(0.95)
        frp_90 = dataset['frp'].quantile(0.90)
        frp_85 = dataset['frp'].quantile(0.85)
        
        # Alternative 1: Top 5% FRP anywhere
        opt1_anomalies = (dataset['frp'] > frp_95).sum()
        
        # Alternative 2: Top 10% FRP in low season (May-Aug)
        opt2_anomalies = ((dataset['frp'] > frp_90) & 
                         (dataset['month'].isin([5, 6, 7, 8]))).sum()
        
        # Alternative 3: Top 15% FRP in winter (Jun-Aug)
        opt3_anomalies = ((dataset['frp'] > frp_85) & 
                         (dataset['month'].isin([6, 7, 8]))).sum()
        
        print(f"Anomaly Definition Options:")
        print(f"Option 1 - Top 5% FRP anywhere: {opt1_anomalies} anomalies")
        print(f"Option 2 - Top 10% FRP in low season: {opt2_anomalies} anomalies")
        print(f"Option 3 - Top 15% FRP in winter: {opt3_anomalies} anomalies")
        
        # Choose the option that gives us 30-100 anomalies
        if 30 <= opt1_anomalies <= 100:
            dataset['is_anomaly'] = (dataset['frp'] > frp_95).astype(int)
            chosen_def = f"Top 5% FRP anywhere (threshold: {frp_95:.2f})"
        elif 30 <= opt2_anomalies <= 100:
            dataset['is_anomaly'] = ((dataset['frp'] > frp_90) & 
                                   (dataset['month'].isin([5, 6, 7, 8]))).astype(int)
            chosen_def = f"Top 10% FRP in low season (threshold: {frp_90:.2f})"
        elif 30 <= opt3_anomalies <= 100:
            dataset['is_anomaly'] = ((dataset['frp'] > frp_85) & 
                                   (dataset['month'].isin([6, 7, 8]))).astype(int)
            chosen_def = f"Top 15% FRP in winter (threshold: {frp_85:.2f})"
        else:
            # Code to choose the option with most anomalies.
            if opt1_anomalies >= opt2_anomalies and opt1_anomalies >= opt3_anomalies:
                dataset['is_anomaly'] = (dataset['frp'] > frp_95).astype(int)
                chosen_def = f"Top 5% FRP anywhere (threshold: {frp_95:.2f})"
            elif opt2_anomalies >= opt3_anomalies:
                dataset['is_anomaly'] = ((dataset['frp'] > frp_90) & 
                                       (dataset['month'].isin([5, 6, 7, 8]))).astype(int)
                chosen_def = f"Top 10% FRP in low season (threshold: {frp_90:.2f})"
            else:
                dataset['is_anomaly'] = ((dataset['frp'] > frp_85) & 
                                       (dataset['month'].isin([6, 7, 8]))).astype(int)
                chosen_def = f"Top 15% FRP in winter (threshold: {frp_85:.2f})"
    
    anomaly_count = dataset['is_anomaly'].sum()
    anomaly_pct = anomaly_count / len(dataset) * 100
    
    print(f"Final Anomaly Definition: {chosen_def}")
    print(f"Dataset Summary:")
    print(f"Total samples: {len(dataset):,}")
    print(f"Anomalous fires: {anomaly_count} ({anomaly_pct:.2f}%)")
    print(f"Normal fires: {len(dataset) - anomaly_count} ({100-anomaly_pct:.2f}%)")
    
    return dataset, chosen_def

# Create the dataset
dataset, anomaly_definition = create_anomaly_dataset()


Working with 8,140 fire samples
Anomaly Definition Options:
Option 1 - Top 5% FRP anywhere: 407 anomalies
Option 2 - Top 10% FRP in low season: 35 anomalies
Option 3 - Top 15% FRP in winter: 6 anomalies
Final Anomaly Definition: Top 10% FRP in low season (threshold: 79.54)
Dataset Summary:
Total samples: 8,140
Anomalous fires: 35 (0.43%)
Normal fires: 8105 (99.57%)


In [22]:
# Data preparation for anomaly detection
def prepare_anomaly_data(data):
    """
    Prepares data for both supervised and unsupervised anomaly detection
    """
    
    # Select features for modeling
    feature_columns = [
        'frp', 'brightness', 'confidence', 'month',
        'temp_max', 'temp_min', 'humidity_min', 'humidity_max',
        'wind_speed', 'precipitation', 'solar_radiation',
        'temp_range', 'humidity_range'
    ]
    
    X = data[feature_columns].copy()
    y = data['is_anomaly'].copy()
    
    # Filling missing values with mean of the column.
    X = X.fillna(X.mean())
    
    print(f"Features: {feature_columns}")
    print(f"Dataset shape: {X.shape}")
    print(f"Anomaly distribution: {y.value_counts().to_dict()}")
    
    # (80/20) Data split.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Scales features for anomaly detection
    # StandardScaler scales features to have mean 0 and std 1.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Converts scaled data back to DataFrames for easier handling.
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_columns)
    
    print(f"Training set: {X_train_scaled.shape[0]} samples ({y_train.sum()} anomalies)")
    print(f"Test set: {X_test_scaled.shape[0]} samples ({y_test.sum()} anomalies)")
    
    # Calculates class weights for extreme imbalance
    # This is used to heavily penalize wrong predictions for minority class.
    if y_train.sum() > 0:  # Only if we have anomalies
        # classes are the unique values in y_train.
        # compute_class_weight is used to calculate class weights for extreme imbalance.
        # 'balanced' is used to calculate class weights based on the class distribution.
        classes = np.unique(y_train)
        class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
        class_weight_dict = dict(zip(classes, class_weights))
        print(f"Class weights for extreme imbalance: {class_weight_dict}")
    else:
        class_weight_dict = None
    
    return X_train_scaled, X_test_scaled, y_train, y_test, feature_columns, scaler, class_weight_dict

# Prepare the data
X_train, X_test, y_train, y_test, features, scaler, class_weights = prepare_anomaly_data(dataset)


Features: ['frp', 'brightness', 'confidence', 'month', 'temp_max', 'temp_min', 'humidity_min', 'humidity_max', 'wind_speed', 'precipitation', 'solar_radiation', 'temp_range', 'humidity_range']
Dataset shape: (8140, 13)
Anomaly distribution: {0: 8105, 1: 35}
Training set: 6512 samples (28 anomalies)
Test set: 1628 samples (7 anomalies)
Class weights for extreme imbalance: {0: 0.5021591610117212, 1: 116.28571428571429}


In [23]:
# Train anomaly detection models
def train_anomaly_models():
    """
    Train both supervised and unsupervised anomaly detection models
    """
    
    models = {}
    results = {}
    
    # UNSUPERVISED MODELS (Best for extreme imbalance)
    print("\nUNSUPERVISED ANOMALY DETECTION: ")
    
    # 1. Isolation Forest (My first choice)
    print("Training Isolation Forest: ")
    contamination = y_train.mean()  # Expected fraction of anomalies
    isolation_forest = IsolationForest(
        contamination=max(0.01, contamination),  # At least 1%
        random_state=42,
        n_estimators=100,
        n_jobs=-1
    )
    
    # Fit on training data (unsupervised)
    isolation_forest.fit(X_train)
    
    # Make predictions (-1 for anomaly, 1 for normal)
    if_train_pred = isolation_forest.predict(X_train)
    if_test_pred = isolation_forest.predict(X_test)
    
    # Convert to binary (0 for normal, 1 for anomaly)
    if_train_binary = (if_train_pred == -1).astype(int)
    if_test_binary = (if_test_pred == -1).astype(int)
    
    models['Isolation Forest'] = isolation_forest
    results['Isolation Forest'] = {
        'test_predictions': if_test_binary,
        'train_predictions': if_train_binary,
        'type': 'unsupervised'
    }
    
    print(f"Isolation Forest trained")
    print(f"Detected {if_test_binary.sum()} anomalies in test set")
    
    # 2. One-Class SVM
    print("Training One-Class SVM: ")
    one_class_svm = OneClassSVM(
        nu=max(0.01, contamination),  # Expected fraction of outliers
        kernel='rbf',
        gamma='scale'
    )
    
    one_class_svm.fit(X_train)
    
    oc_train_pred = one_class_svm.predict(X_train)
    oc_test_pred = one_class_svm.predict(X_test)
    
    oc_train_binary = (oc_train_pred == -1).astype(int)
    oc_test_binary = (oc_test_pred == -1).astype(int)
    
    models['One-Class SVM'] = one_class_svm
    results['One-Class SVM'] = {
        'test_predictions': oc_test_binary,
        'train_predictions': oc_train_binary,
        'type': 'unsupervised'
    }
    
    print(f"One-Class SVM trained")
    print(f"Detected {oc_test_binary.sum()} anomalies in test set")
    
    # 3. Local Outlier Factor (for novelty detection)
    print("Training Local Outlier Factor: ")
    lof = LocalOutlierFactor(
        contamination=max(0.01, contamination),
        novelty=True  # For predicting on new data
    )
    
    lof.fit(X_train)
    
    lof_train_pred = lof.predict(X_train)
    lof_test_pred = lof.predict(X_test)
    
    lof_train_binary = (lof_train_pred == -1).astype(int)
    lof_test_binary = (lof_test_pred == -1).astype(int)
    
    models['Local Outlier Factor'] = lof
    results['Local Outlier Factor'] = {
        'test_predictions': lof_test_binary,
        'train_predictions': lof_train_binary,
        'type': 'unsupervised'
    }
    
    print(f"Local Outlier Factor trained")
    print(f"Detected {lof_test_binary.sum()} anomalies in test set")
    
    # SUPERVISED MODELS (Only if we have atleast 5 anomalies, then we can use Random Forest and
    # Logistic Regression models that we had tried earlier.)
    if y_train.sum() >= 5:  # Only if we have at least 5 anomalies
        print("\nSUPERVISED MODELS WITH EXTREME CLASS BALANCING: ")
        
        # 4. Random Forest with extreme class balancing (due to the low outlier count)
        print("Training Random Forest (Class Balanced): ")
        rf_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            class_weight='balanced_subsample',  # To handle extreme imbalance
            random_state=42,
            n_jobs=-1
        )
        
        rf_model.fit(X_train, y_train)
        
        rf_train_pred = rf_model.predict(X_train)
        rf_test_pred = rf_model.predict(X_test)
        
        models['Random Forest'] = rf_model
        results['Random Forest'] = {
            'test_predictions': rf_test_pred,
            'train_predictions': rf_train_pred,
            'feature_importance': rf_model.feature_importances_,
            'type': 'supervised'
        }
        
        print(f"Random Forest trained")
        print(f"Predicted {rf_test_pred.sum()} anomalies in test set")
        
        # 5. Logistic Regression with extreme class balancing
        print("Training Logistic Regression (Class Balanced): ")
        lr_model = LogisticRegression(
            class_weight='balanced',
            random_state=42,
            max_iter=1000,
            C=0.1  # Stronger regularization for small dataset, follows rules very strictly to prevent overfitting.
        )
        
        lr_model.fit(X_train, y_train)
        
        lr_train_pred = lr_model.predict(X_train)
        lr_test_pred = lr_model.predict(X_test)
        
        models['Logistic Regression'] = lr_model
        results['Logistic Regression'] = {
            'test_predictions': lr_test_pred,
            'train_predictions': lr_train_pred,
            'type': 'supervised'
        }
        
        print(f"Logistic Regression trained")
        print(f"Predicted {lr_test_pred.sum()} anomalies in test set")
    
    else:
        print("Skipping supervised models since there are too few anomalies for reliable training")
        print("Recommendedse unsupervised methods only")
    
    print("\nAll applicable models trained successfully!")
    return models, results

# Train all models
models, model_results = train_anomaly_models()



UNSUPERVISED ANOMALY DETECTION: 
Training Isolation Forest: 
Isolation Forest trained
Detected 20 anomalies in test set
Training One-Class SVM: 
One-Class SVM trained
Detected 38 anomalies in test set
Training Local Outlier Factor: 
Local Outlier Factor trained
Detected 19 anomalies in test set

SUPERVISED MODELS WITH EXTREME CLASS BALANCING: 
Training Random Forest (Class Balanced): 
Random Forest trained
Predicted 7 anomalies in test set
Training Logistic Regression (Class Balanced): 
Logistic Regression trained
Predicted 74 anomalies in test set

All applicable models trained successfully!


In [ ]:
# Evaluate anomaly detection models
from IPython.display import display
def evaluate_anomaly_models():
    """
    Evaluates and compares all the anomaly detection models
    """
    
    # Calculates metrics for models with ground truth class labels. (y_test)
    evaluation_results = {}
    
    for model_name, results in model_results.items():
        test_preds = results['test_predictions']
        
        # Calculate metrics
        if len(np.unique(y_test)) > 1 and len(np.unique(test_preds)) > 1:
            # Only calculate when both classes exist
            precision = precision_score(y_test, test_preds, zero_division=0)
            recall = recall_score(y_test, test_preds, zero_division=0)
            f1 = f1_score(y_test, test_preds, zero_division=0)
            accuracy = accuracy_score(y_test, test_preds)
        else:
            precision = recall = f1 = accuracy = 0.0
        
        evaluation_results[model_name] = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'accuracy': accuracy,
            'detected_anomalies': test_preds.sum(),
            'actual_anomalies': y_test.sum()
        }
    
    # Creates a comparison table
    print("\nMODEL COMPARISON")
    comparison_data = []
    for model_name, metrics in evaluation_results.items():
        comparison_data.append({
            'Model': model_name,
            'Type': model_results[model_name]['type'],
            'Precision': f"{metrics['precision']:.3f}",
            'Recall': f"{metrics['recall']:.3f}",
            'F1-Score': f"{metrics['f1_score']:.3f}",
            'Detected': metrics['detected_anomalies'],
            'Actual': metrics['actual_anomalies']
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    display(comparison_df)
    
    # Prints detailed results for each model
    for model_name, results in model_results.items():
        print(f"\n{model_name.upper()} DETAILED RESULTS: ")
        test_preds = results['test_predictions']
        
        print("Classification Report:")
        if len(np.unique(y_test)) > 1 and len(np.unique(test_preds)) > 1:
            report = classification_report(y_test, test_preds, zero_division=0)
            print(report)
            
            # Confusion matrix
            cm = confusion_matrix(y_test, test_preds)
            print(f"\nConfusion Matrix:")
            print(f"              Predicted")
            print(f"              Normal  Anomaly")
            print(f"Actual Normal   {cm[0,0] if cm.shape[0] > 0 else 0:4d}    {cm[0,1] if cm.shape[0] > 1 else 0:4d}")
            if cm.shape[0] > 1:
                print(f"       Anomaly  {cm[1,0]:4d}    {cm[1,1]:4d}")
        else:
            print("Cannot generate classification report since there is insufficient class diversity")
    
    return evaluation_results

# Run evaluation
evaluation_results = evaluate_anomaly_models()



MODEL COMPARISON


,Model,Type,Precision,Recall,F1-Score,Detected,Actual
0,Isolation Forest,unsupervised,0.000,0.000,0.000,20,7
1,One-Class SVM,unsupervised,0.026,0.143,0.044,38,7
2,Local Outlier Factor,unsupervised,0.000,0.000,0.000,19,7
3,Random Forest,supervised,0.857,0.857,0.857,7,7
4,Logistic Regression,supervised,0.081,0.857,0.148,74,7



ISOLATION FOREST DETAILED RESULTS: 
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1621
           1       0.00      0.00      0.00         7

    accuracy                           0.98      1628
   macro avg       0.50      0.49      0.50      1628
weighted avg       0.99      0.98      0.99      1628


Confusion Matrix:
              Predicted
              Normal  Anomaly
Actual Normal   1601      20
       Anomaly     7       0

ONE-CLASS SVM DETAILED RESULTS: 
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1621
           1       0.03      0.14      0.04         7

    accuracy                           0.97      1628
   macro avg       0.51      0.56      0.52      1628
weighted avg       0.99      0.97      0.98      1628


Confusion Matrix:
              Predicted
              Normal  Anomaly
Actual Normal   1584     